In [ ]:
import subprocess
import configparser

import matplotlib.pyplot as plt
import pickle
import numpy as np
import os

if not os.path.isdir('./graphs'):
    os.makedirs('./graphs')

import math

#plt.style.use("ggplot")

class Trader:
    pass

class CustomUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if name == 'Trader':
            return Trader
        return super().find_class(module, name)

In [ ]:
def plot_expectations(filename):
    data = CustomUnpickler(open(os.path.join('./output', filename),'rb')).load()
    #trades = data['assets'][0].trades
    #print(len(trades))
    #print(sum([trade.buyer == trade.seller for trade in trades]))
    #fig, axs = plt.subplots(4, sharex=True, sharey=True)
    for i in range(0,4):
        names = []
        for j in range(0,4):
            agent = data['agents'][j]
            if(hasattr(agent, 'expectations')):
                expectations = agent.expectations
                plt.plot(expectations[:,0],expectations[:,i+1])
                names.append(agent.name)
        plt.ylabel('value')
        plt.xlabel('time')
        plt.legend(labels = names)
        titlestring = 'Agents\' view of suit '+str(i)+'\'s value over time'
        if i == data['goalsuit']:
            titlestring += " (GOAL SUIT)"
        plt.title(titlestring)
        plt.show()

In [ ]:
def plot_rewards(input_base, iterations, output):
    endcash = np.zeros([iterations,4])
    endpayout = np.zeros([iterations,4])
    avg_trades = 0
    for i in range(0, iterations):
        filename = os.path.join('./output',input_base+'.'+str(i))
        data = CustomUnpickler(open(filename, 'rb')).load()
        cash = [agent.cash for agent in data['agents']]
        payout = [agent.payout for agent in data['agents']]
        names = [agent.name for agent in data['agents']]
        endcash[i,:] = cash
        endpayout[i,:] = payout
        avg_trades += sum([len(asset.trades) for asset in data["assets"]])/iterations
    cashmeans = np.mean(endcash,axis=0)
    payoutmeans = np.mean(endpayout,axis=0)
    print(cashmeans)
    cashstds  = np.std(endcash,axis=0)
    payoutstds  = np.std(endpayout,axis=0)
    print(payoutmeans)
    #print(endpayout)
    #print(payoutstds)

    fig, ax = plt.subplots()
    color = [agent.color for agent in data['agents']]
    ax.bar(range(0,4),cashmeans,color = color, edgecolor = "black")
    ax.errorbar(range(0,4), cashmeans, yerr = cashstds*2/math.sqrt(iterations),fmt="none", color="grey",capsize=5)
    ax.bar(range(0,4),payoutmeans,color = color,bottom=cashmeans, edgecolor = "black")
    ax.errorbar(range(0,4), payoutmeans+cashmeans, yerr=payoutstds*2/math.sqrt(iterations), fmt="none", color="black",capsize=5)
    ax.set_ylim((0,600))
    ax.set_ylabel("Mean cash + payout at end")
    ax.set_xlabel("Player")
    ax.set_xticks(range(0,4))
    ax.set_xticklabels(names)

    plt.show()
    fig.savefig('graphs/'+output+".png", facecolor='white', transparent=False)

In [ ]:
iterations = 100

In [ ]:
for i in range(1, 4):
    print(i)
    config = configparser.ConfigParser()
    config['fundamental'] = {'source': 'fundamentalist',
                             'n':i,
                             'cash': 350}
    prey = [k for k in range(0,i)]
    config['bottomfeeder'] = {'source': 'bottomfeeder_v2',
                              'n':1,
                              'cash': 350,
                              'params':{'prey': prey, 'ordersno': 4}}
    config['noisy'] = {'source': 'noise',
                       'n':4-i-1,
                       'cash': 350,
                       'params': '{\'var\': 1, \'horizon\': 50}'}
    with open('configs/temp_config.ini', 'w') as configfile:
        config.write(configfile)
    for j in range(0, iterations):
        subprocess.run(['./market-des', '-r figgie -s 10000', '--index '+str(j),
               '--output '+'bvsf.'+str(i), '--config '+"temp_config"],stdout=subprocess.DEVNULL)


In [ ]:
for i in range(1, 4):
    filename = 'bvsf'+'.'+str(i)
    print(i)
    plot_rewards(filename, iterations, filename)

In [ ]:
configs = ['4n','1f3n','2f2n','3f1n','4f']
iterations = 100

In [ ]:
for config in configs:
    print(config)
    for i in range(0, iterations):
        subprocess.run(['./market-des', '-r figgie -s 10000', '--index '+str(i),
                       '--output '+config, '--config '+config],stdout=subprocess.DEVNULL)

In [ ]:
for config in configs:
    print(config)
    plot_rewards(config, iterations, config)

In [ ]:
data = CustomUnpickler(open(os.path.join('./output', '1f3n.4'),'rb')).load()
data = data['agents'][0].expectations
data = data[1:100,:]
for i in range(1,5):
    plt.plot(data[:,0],data[:,i])
plt.ylabel('value')
plt.xlabel('time')
plt.legend(labels = range(0,4))
plt.title('Fundamentalist\' view of suit\'s value over time')
plt.show()

In [ ]:
data = CustomUnpickler(open(os.path.join('./output', '1f3n.4'),'rb')).load()
for i in range(0,4):
    asset = data['assets'][i]
    price_series = asset.price_series
    times = price_series[:,0]
    prices = price_series[:,1]
    plt.plot(times,prices)
plt.ylabel('price')
plt.xlabel('time')
plt.title('Asset prices over time')
plt.legend(labels = range(0,4))
plt.show()

In [ ]:
iterations=100
config = 'bottomfeeders'
for i in range(0, iterations):
    subprocess.run(['./market-des', '-r figgie -s 10000', '--index '+str(i),
                   '--output '+config, '--config '+config],stdout=subprocess.DEVNULL)

In [ ]:
plot_rewards('bottomfeeders',iterations,"bottomfeeders")

In [ ]:
iterations=100
config = 'bottomfeeders_v2'
for i in range(0, iterations):
    subprocess.run(['./market-des', '-r figgie -s 10000', '--index '+str(i),
                   '--output '+config, '--config '+config],stdout=subprocess.DEVNULL)

In [ ]:
plot_rewards('bottomfeeders_v2',iterations,"bottomfeeders_v2")

In [ ]:
#     orders = data['agents'][0].orders
#     orders = [order for order in orders if order.assetno == 0]
#     order_prices = [order.price for order in orders]
#     colors = ["red" if order.buy else "green" for order in orders]
#     plt.scatter(range(0,len(orders)),order_prices,color = colors)
#     plt.show()
plot_expectations('bottomfeeders_v2.2')

In [ ]:
plot_expectations('3f1n.0')

In [ ]:
# multiple first-degrees looking at fundamentalist